In [ ]:
import os
from pymetabo.core import *
from pymetabo.helpers import *
from pymetabo.dataframes import *

Specifiy input mzML, result and interim directories

In [ ]:
mzML_dir = "data/mzML"
results = "results"
interim = Helper().reset_directory(os.path.join(results, "interim"))

In [ ]:
for mzml in os.listdir(mzML_dir):
    print(mzml)

Precursor Correction to highest intensity

In [ ]:
PrecursorCorrector().to_highest_intensity(mzML_dir, os.path.join(interim, "mzML_PCpeak"))

Feature detection by FFM

In [ ]:

FeatureFinderMetabo().run(os.path.join(interim, "mzML_PCpeak"), os.path.join(interim, "FFM"),
                        {"noise_threshold_int": 10000.0,
                        "mass_error_ppm": 10.0,
                        "remove_single_traces": "true"})

Precursor Correction to nearest feature


In [ ]:
PrecursorCorrector().to_nearest_feature(os.path.join(interim, "mzML_PCpeak"),
                                        os.path.join(interim, "mzML_PCfeature"),
                                        os.path.join(interim, "FFM"))

Map alignment of FetureMaps and mzML files

In [ ]:
MapAligner().run(os.path.join(interim, "FFM"), os.path.join(interim, "FFM_aligned"),
                os.path.join(interim, "Trafo"),
                {"max_num_peaks_considered": -1,
                "superimposer:mz_pair_max_distance": 0.05,
                "pairfinder:distance_MZ:max_difference": 10.0,
                "pairfinder:distance_MZ:unit": "ppm"})

MapAligner().run(os.path.join(interim, "mzML_PCfeature"), os.path.join(interim, "mzML_aligned"),
                os.path.join(interim, "Trafo"))

Feature linking

In [ ]:
FeatureLinker().run(os.path.join(interim, "FFM_aligned"),
                os.path.join(interim,  "FFM.consensusXML"))

DataFrames().create_consensus_table(os.path.join(
    interim, "FFM.consensusXML"), os.path.join(interim, "FFM_consensus.tsv"))

Requantification with FFMID

In [ ]:
FeatureMapHelper().split_consensus_map(os.path.join(interim,  "FFM.consensusXML"),
                                    os.path.join(interim,  "FFM_complete.consensusXML"),
                                    os.path.join(interim,  "FFM_missing.consensusXML"))

FeatureMapHelper().consensus_to_feature_maps(os.path.join(interim,  "FFM_complete.consensusXML"),
                                            os.path.join(interim, "FFM_aligned"),
                                            os.path.join(interim, "FFM_complete"))

FeatureFinderMetaboIdent().run(os.path.join(interim, "mzML_aligned"),
                            os.path.join(interim,  "FFMID"),
                            os.path.join(interim,  "FFM_missing.consensusXML"),
                            {"detect:peak_width": 60.0})

FeatureMapHelper().merge_feature_maps(os.path.join(interim, "FeatureMaps_merged"), os.path.join(
    interim, "FFM_complete"), os.path.join(interim, "FFMID"))

Adduct decharging 

In [ ]:
MetaboliteAdductDecharger().run(os.path.join(interim, "FeatureMaps_merged"), os.path.join(interim, "FeatureMaps_decharged"),
                            {"potential_adducts": [b"H:+:0.5", b"Na:+:0.3", b"H-1O-1:+:0.2"],
                            "charge_min": 1,
                            "charge_max": 3,
                            "max_neutrals": 2})

Map MS2 spectra to features

In [ ]:

MapID().run(os.path.join(interim, "mzML_aligned"), os.path.join(
    interim, "FeatureMaps_decharged"), os.path.join(interim, "FeatureMaps_ID_mapped"))

Link re-quantified features to final ConsensusMap

In [ ]:
FeatureLinker().run(os.path.join(interim, "FeatureMaps_ID_mapped"),
                os.path.join(interim, "FeatureMatrix.consensusXML"))

DataFrames().create_consensus_table(os.path.join(
    interim, "FeatureMatrix.consensusXML"), os.path.join(results, "FeatureMatrix.tsv"))

In [ ]:
Sirius().run()